## **Составление таблиц**

In [4]:
import numpy as np
import pandas as pd

### Произведение:

● id

● название

In [5]:
# Таблица из 50 различных книг
titles_qty = 50
book_id = np.arange(1, titles_qty+1)
titles = [f'title_{n}' for n in range(1, titles_qty+1)]
titles[:5]

['title_1', 'title_2', 'title_3', 'title_4', 'title_5']

In [6]:
book = pd.DataFrame({
    'id': book_id,
    'title': titles})

book.head()

,id,title
0,1,title_1
1,2,title_2
2,3,title_3
3,4,title_4
4,5,title_5


### Издание:

● id

● id_произведения

● год издания

● кол-во страниц

In [7]:
# Таблица из 200 изданий для 45 книг(5 оставляем непривязанными), с 90 по 22 год, размерами книг от 40 до 340 страниц
np.random.seed(42)

edition_qty = 200
edition_id = np.arange(1, edition_qty+1)
id_book = np.random.choice(book_id[5:], edition_qty)  # first 5 without edition
year_edition = np.random.randint(1990, 2022, size=edition_qty)
pages_qty = np.random.randint(40, 340, size=edition_qty)

In [8]:
edition = pd.DataFrame({
    'id': edition_id,
    'id_book': id_book,
    'year_edition': year_edition,
    'pages_qty': pages_qty,
    })

edition.head()

,id,id_book,year_edition,pages_qty
0,1,44,2016,220
1,2,34,2006,134
2,3,20,1991,138
3,4,48,1991,227
4,5,13,2017,155


### Экземпляр

● id

● id_издание

● инвентаризационный номер

In [9]:
# Таблица из 2000 экземпляров книг, их изданий(несколько непривязанных с пропусками) и инвент.номера начиная с 25001
exemplar_qty = 2000
exemplar_id = np.arange(1, exemplar_qty+1)
id_edition = np.random.choice(np.append(edition_id, np.NaN), exemplar_qty)

inventory_start_n = 25001
inventory_number = np.arange(inventory_start_n, inventory_start_n+exemplar_qty)

In [10]:
exemplar = pd.DataFrame({
    'id': exemplar_id,
    'id_edition': id_edition,
    'inventory_number': inventory_number
    }, dtype='Int64')

exemplar.head()

,id,id_edition,inventory_number
0,1,151,25001
1,2,144,25002
2,3,57,25003
3,4,39,25004
4,5,109,25005


### Лог операций

● id

● id_user

● id_экземпляр

● дата_взяли

● дата_вернули

In [11]:
# Функции для генерации дат

# книга берется в случайный день 2021го года
def generate_random_day(start_date, range_in_days):
    days_to_add = np.random.randint(1, range_in_days)
    random_date = pd.to_datetime(start_date) + pd.DateOffset(days=days_to_add)
    return random_date

# книга возращается с вероятностью 0.95, спустя случайное кол-во дней от 1 до 100
def generate_return_day(date_taken, return_range=100):  
    days_to_add = np.random.randint(1, return_range)
    return_prob = 0.95
    if np.random.binomial(1, return_prob, 1)[0]:
        date_return = pd.to_datetime(date_taken) + pd.DateOffset(days=days_to_add)
    else:
        date_return = np.NaN
    return date_return

In [12]:
# Таблица из 500 операций на 150 пользователей, начиная с 2021-01-01
start_day = '2021-01-01'
action_qty = 500
user_qty = 150

id_log = np.arange(1, action_qty+1)
id_user = np.random.choice(user_qty, action_qty)
id_exemplar = np.random.choice(exemplar_id, action_qty)

date_taken = [generate_random_day(start_day, 365) for _ in range(action_qty)]
date_returned = [generate_return_day(day) for day in date_taken]

In [13]:
log_action = pd.DataFrame({
    'id': id_log,
    'id_user': id_user,
    'id_exemplar': id_exemplar,
    'date_taken': date_taken,
    'date_returned': date_returned, 
    })

log_action.head()

,id,id_user,id_exemplar,date_taken,date_returned
0,1,94,40,2021-10-29,NaT
1,2,13,486,2021-12-24,2022-02-13
2,3,64,929,2021-08-02,2021-10-16
3,4,109,220,2021-03-04,2021-05-30
4,5,79,1153,2021-02-19,2021-05-02


## **SQL Часть**

In [1]:
# installing sqlite for colab 
!pip install pandasql
from pandasql import sqldf
from tabulate import tabulate
import sqlite3
sqlite3.sqlite_version  # должна быть от '3.36.0'

'3.36.0'

In [ ]:
# обновляем версию sqlite для использования оконок и т.п.
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
# Необходимо перезагрузить Runtime после обновления!

In [2]:
# Произведение - book
# Издание - edition
# Экземпляр - exemplar
# Лог_операций - log_action

In [14]:
print('Произведение(book)', book.shape)
print(tabulate(book.head(), headers='keys', tablefmt='psql'))

Произведение(book) (50, 2)
+----+------+---------+
|    |   id | title   |
|----+------+---------|
|  0 |    1 | title_1 |
|  1 |    2 | title_2 |
|  2 |    3 | title_3 |
|  3 |    4 | title_4 |
|  4 |    5 | title_5 |
+----+------+---------+


In [15]:
print('Издание(edition)', edition.shape)
print(tabulate(edition.head(), headers='keys', tablefmt='psql'))

Издание(edition) (200, 4)
+----+------+-----------+----------------+-------------+
|    |   id |   id_book |   year_edition |   pages_qty |
|----+------+-----------+----------------+-------------|
|  0 |    1 |        44 |           2016 |         220 |
|  1 |    2 |        34 |           2006 |         134 |
|  2 |    3 |        20 |           1991 |         138 |
|  3 |    4 |        48 |           1991 |         227 |
|  4 |    5 |        13 |           2017 |         155 |
+----+------+-----------+----------------+-------------+


In [16]:
print('Экземпляр(exemplar)', exemplar.shape)
print(tabulate(exemplar.head(), headers='keys', tablefmt='psql'))

Экземпляр(exemplar) (2000, 3)
+----+------+--------------+--------------------+
|    |   id |   id_edition |   inventory_number |
|----+------+--------------+--------------------|
|  0 |    1 |          151 |              25001 |
|  1 |    2 |          144 |              25002 |
|  2 |    3 |           57 |              25003 |
|  3 |    4 |           39 |              25004 |
|  4 |    5 |          109 |              25005 |
+----+------+--------------+--------------------+


In [17]:
print('Лог_операций(log_action)', log_action.shape)
print(tabulate(log_action.head(), headers='keys', tablefmt='psql'))

Лог_операций(log_action) (500, 5)
+----+------+-----------+---------------+---------------------+---------------------+
|    |   id |   id_user |   id_exemplar | date_taken          | date_returned       |
|----+------+-----------+---------------+---------------------+---------------------|
|  0 |    1 |        94 |            40 | 2021-10-29 00:00:00 | NaT                 |
|  1 |    2 |        13 |           486 | 2021-12-24 00:00:00 | 2022-02-13 00:00:00 |
|  2 |    3 |        64 |           929 | 2021-08-02 00:00:00 | 2021-10-16 00:00:00 |
|  3 |    4 |       109 |           220 | 2021-03-04 00:00:00 | 2021-05-30 00:00:00 |
|  4 |    5 |        79 |          1153 | 2021-02-19 00:00:00 | 2021-05-02 00:00:00 |
+----+------+-----------+---------------+---------------------+---------------------+


### **1) Найти произведения, которые издавались более 5 раз**

In [18]:
query_1 = """
SELECT 
  b.title
FROM 
  edition AS e
LEFT JOIN book AS b 
  ON e.id_book = b.id
GROUP BY 
  b.title
HAVING 
  COUNT(b.title) > 5;
"""

sqldf(query_1)

,title
0,title_13
1,title_14
2,title_19
3,title_20
4,title_29
5,title_33
6,title_40
7,title_42
8,title_44
9,title_47


### **2) Проверить, есть ли экземпляры, не привязанные к ни к одному изданию.**

In [19]:
query_2 = """
SELECT 
  id
FROM
  exemplar
WHERE 
  id_edition IS Null;
"""

sqldf(query_2)

,id
0,131
1,184
2,636
3,808
4,885
5,889
6,1565
7,1834
8,1952


### **3) Для каждого пользователя найти последние три взятые им произведения. Для каждого такого произведения указать сколько всего раз ее брали (за все время).**

In [22]:
# 1) в journal для каждого юзера ранжируем взятые книги по дате(+ джойним чтобы вытащить название произведения)
# 2) в taken_times считаем сколько раз взята книга, и джойним счет в журнал, к-й фильтруем по посл.3м книгам
query_3 = """
WITH journal AS (
SELECT la.id_user, la.date_taken, b.title,
  ROW_NUMBER() OVER (PARTITION BY la.id_user ORDER BY la.date_taken DESC) AS rn
FROM 
  log_action AS la
LEFT JOIN 
  exemplar AS ex 
  ON la.id_exemplar = ex.id
LEFT JOIN 
  edition AS ed 
  ON ex.id_edition = ed.id 
LEFT JOIN 
  book AS b 
  ON ed.id_book = b.id 
    ),

taken_times	AS (
SELECT 
  title, 
  COUNT(date_taken) AS times_taken 
FROM 
  journal 
GROUP BY 
  title
)
	
SELECT 
  j.id_user, 
  j.title, 
  t.times_taken
FROM 
  journal AS j
LEFT JOIN 
  taken_times AS t 
  ON j.title = t.title
WHERE 
  rn <= 3;
"""


sqldf(query_3)

,id_user,title,times_taken
0,1,title_38,23.0
1,1,title_50,20.0
2,1,title_14,19.0
3,2,title_49,29.0
4,2,title_26,11.0
...,...,...,...
370,148,title_38,23.0
371,148,title_17,7.0
372,148,title_21,6.0
373,149,title_11,5.0


### **4) Список самых неблагонадежных пользователей библиотеки – рейтинг 10 самыхсамых плохих пользователей по двум или более критериям. Критерии неблагонадежности, с точки зрения бизнеса, предложите самостоятельно.**

Взял ванильный подход, рассчитывающий потенциальный ущерб от клиента, исходя из времени хранения у себя взятых книг с учетом их размера. (Чем больше книга - тем больше ценности для библиотеки -> чем на дольше юзер забирает бо́льшую ценность у библиотеки - тем он для неё "вреднее")


In [23]:
# в holding собираем для каждого юзера инфу о времени хранения и объеме взятых книг (
# 		текущей датой считаем дату посл.возврата, + заполняем пропуски в date_returned и pages_qty)
# каждый юзер получает усредненые отмасштабированые(minmax) признаки времени хранения и объема книг, 
#			произведение которых будет "коэффициентом плохости" клиента для бизнеса. Топ 10.

query_4 = """
WITH holding AS (
SELECT
	la.id_user,
	round(COALESCE(ed.pages_qty, AVG(ed.pages_qty) OVER())) AS pages_filled,
  JULIANDAY(COALESCE(la.date_returned, datetime(MAX(la.date_returned) OVER() , '+1 day'))) -  JULIANDAY(la.date_taken) AS days_holding	
FROM 
	log_action AS la
LEFT JOIN 
	exemplar AS ex 
	ON la.id_exemplar = ex.id
LEFT JOIN 
	edition AS ed 
	ON ex.id_edition = ed.id 
),

user_features AS (
SELECT 
	id_user,
  AVG(days_holding) AS avg_holding,
  round(AVG(pages_filled)) AS avg_pages  
FROM 
  holding 
GROUP BY 
  id_user
),

features_scaled AS (
SELECT 
	id_user, 
	(avg_holding - MIN(avg_holding) OVER()) / (MAX(avg_holding) OVER() - MIN(avg_holding) OVER()) AS avg_holding_normed,
	(avg_pages - MIN(avg_pages) OVER()) / (MAX(avg_pages) OVER() - MIN(avg_pages) OVER()) AS avg_pages_normed
FROM 
  user_features
)
	
SELECT 
  id_user, 
	round(avg_holding_normed * avg_pages_normed, 3) AS unreliable_coef
FROM 
  features_scaled
ORDER BY 
  unreliable_coef DESC
LIMIT 10
"""

sqldf(query_4)

# postgresql to sqlite corrections:
#  sqlite non_supported interval -> replaced to datetime('', '+1 day')
#  sqlite non_supported "date - date" -> replaced to datediff
#  sqlite non_supported "extract(from day..)" -> removed as unnecessary

,id_user,unreliable_coef
0,98,0.474
1,14,0.436
2,139,0.343
3,71,0.317
4,92,0.289
5,10,0.258
6,90,0.243
7,5,0.237
8,129,0.235
9,53,0.224
